In [1]:
# Imports
import pandas as pd
import numpy as np

In [2]:
# Data filenames
basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'

In [3]:
# Load Data
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

# Filtering/Cleaning

## Replace "\N" (missing values) with np.nan and make them permanent

In [4]:
# Check missing values for all dataframes and replace with NAN
basics.isna().sum()

tconst             0
titleType          0
primaryTitle      11
originalTitle     11
isAdult            0
startYear          0
endYear            0
runtimeMinutes     0
genres            11
dtype: int64

In [5]:
ratings.isna().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

In [6]:
akas.isna().sum()

titleId              0
ordering             0
title                5
region             111
language             0
types                0
attributes           0
isOriginalTitle      0
dtype: int64

In [7]:
basics.replace({'\\N':np.nan}, inplace=True)
akas.replace({'\\N':np.nan}, inplace=True)

## Eliminate movies that are null for runtimeMinutes (in 'basics' df)

In [8]:
# Drop rows in 'basics' df for null in 'runtimeMinutes' column and re-display
basics = basics.dropna(subset=['runtimeMinutes'])

In [9]:
basics['runtimeMinutes'].isna().sum()

0

## Eliminate movies that are null for genre

In [10]:
# Drop rows in 'basics' df for null in 'genre' column and re-display
basics.dropna(subset=['genres'], inplace=True)
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear          161585
endYear           2731308
runtimeMinutes          0
genres                  0
dtype: int64

## Keep only titleType==Movie

In [11]:
# Display all unique values in 'titleType' column
basics['titleType'].value_counts()

tvEpisode       1400096
short            596005
movie            379725
video            179599
tvMovie           91143
tvSeries          89865
tvSpecial         17874
tvMiniSeries      17000
tvShort            8720
videoGame           317
Name: titleType, dtype: int64

In [12]:
# Keep only the 'Movie' category and re-display to confirm
basics = basics[basics['titleType']=='movie']
basics['titleType'].value_counts()

movie    379725
Name: titleType, dtype: int64

## Keep startYear 2000-2022

In [29]:
# Display number of unique values in 'startYear' column
basics['startYear'].nunique()

129

In [36]:
# Keep only years 2000-2022 and re-display values
basics = basics[basics['startYear'].isin(['2000','2001','2002','2003','2004',
                                         '2005','2006','2007','2008','2009',
                                         '2010','2011','2012','2013','2014',
                                         '2015','2016','2017','2018','2019',
                                         '2020','2021','2022'])]
basics['startYear'].value_counts()

2017    14356
2018    14302
2019    14029
2016    13937
2015    13452
2014    13081
2022    12541
2013    12366
2021    12279
2012    11612
2020    11529
2011    10760
2010    10194
2009     9335
2008     8139
2007     6950
2006     6500
2005     5814
2004     5186
2003     4577
2002     4127
2001     3853
2000     3633
Name: startYear, dtype: int64

## Eliminate movies that include "Documentary" in genre

In [39]:
# Exclude movies that are included in the documentary category and re-display value counts
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]
basics['genres'].value_counts()

Drama                         35920
Comedy                        13419
Comedy,Drama                   6449
Horror                         5770
Drama,Romance                  4299
                              ...  
Action,Animation,Game-Show        1
Family,Musical,Sport              1
Horror,Music,Mystery              1
Comedy,History,Mystery            1
Crime,Fantasy,Sci-Fi              1
Name: genres, Length: 969, dtype: int64

## Keep only US movies 

In [40]:
# Filter the 'basics' table down to only include the US by using the filter 'akas' dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers


34803      True
61116      True
67669      True
77964      True
86801      True
           ... 
9722329    True
9722338    True
9722377    True
9722422    True
9722506    True
Name: tconst, Length: 147049, dtype: bool

In [41]:
# Filter 'basics' df
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77964,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
9722329,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
9722338,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"
9722377,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,NaN,84,Thriller
9722422,tt9916362,movie,Coven,Akelarre,0,2020,NaN,92,"Drama,History"


In [45]:
# Keep only US movies in 'ratings' df
# Filter the 'basics' table down to only include the US by using the filter 'akas' dataframe
keepers2 =ratings['tconst'].isin(akas['titleId'])
keepers2

0           True
1           True
2           True
3           True
4           True
           ...  
1294281     True
1294282     True
1294283    False
1294284    False
1294285    False
Name: tconst, Length: 1294286, dtype: bool

In [46]:
# Filter 'ratings' df
ratings = ratings[keepers2]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1965
1,tt0000002,5.8,263
2,tt0000003,6.5,1803
3,tt0000004,5.6,179
4,tt0000005,6.2,2603
...,...,...,...
1294266,tt9916460,9.4,18
1294269,tt9916538,8.6,7
1294270,tt9916544,6.9,62
1294281,tt9916730,8.3,10


In [49]:
# Keep only US movies in 'akas' df
akas = akas[akas['region']=='US']
akas['region']

5           US
14          US
33          US
36          US
41          US
            ..
35380742    US
35380812    US
35380901    US
35380944    US
35380960    US
Name: region, Length: 1424968, dtype: object